# Extraction of OCV curve

In [ ]:
import bdat
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt

Import the data

In [ ]:
# build metadata
batterytype = bdat.BatterySpecies("BAK CG50", capacity=5.0, endOfChargeVoltage=4.2, endOfDischargeVoltage=2.5)
battery = bdat.Battery("DigiBatt_CG50_1072", type=batterytype)
cycling = bdat.Cycling("checkup example", object=battery)

df = pd.read_parquet("1844272_DigiBatt_Checkup.parquet")
dataspec = bdat.get_dataspec(cycling, df)
data = bdat.CyclingData(cycling, df, dataspec)

Segment cycling data into steps

In [ ]:
# find steps
steplist = bdat.steps.find_steps(data)

# plot steps
bdat.plots.plot_steps(None, steplist, data.df, dataspec).show()

In [ ]:
# specify patterns to look for
patterntypes = [bdat.patterns.ChargeQOCV(), bdat.patterns.DischargeQOCV()]

# search the test steps for matches
testeval = bdat.patterns.find_patterns(steplist, patterntypes, data)

# plot results
bdat.plots.plot_testevals(None, testeval, data.df, mark_whole_match=False).show()

Plot charge and discharge QOCV over SOC

In [ ]:
for eval in testeval.evals:
    plt.plot(eval.socNominal, eval.voltage)
    plt.xlabel("SOC / %")
    plt.ylabel("voltage / V")

Write results to CSV files

In [ ]:
for eval in testeval.evals:
    if isinstance(eval, bdat.entities.patterns.ChargeQOCVEval):
        pd.DataFrame({"soc": eval.socNominal, "voltage": eval.voltage}).to_csv("qocv_charge.csv", index=False)
    elif isinstance(eval, bdat.entities.patterns.DischargeQOCVEval):
        pd.DataFrame({"soc": eval.socNominal, "voltage": eval.voltage}).to_csv("qocv_discharge.csv", index=False)